In [1]:
from IPython.display import display
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from qutip import Bloch, Qobj
import time
import io

In [2]:
# Generate data
def rotate_X(θ):
    return np.array([[np.cos(θ/2), 1j*np.sin(θ/2)],[1j*np.sin(θ/2),np.cos(θ/2)]])

def rotate_Z(θ):
    return np.array([[np.exp(1j*θ/2), 0],[0, np.exp(-1j*θ/2)]])


In [5]:
def ramsey_factory(rot_angle):
    states = [np.array([[1.],[0.]])]
    Npihalf=5
    dX = rotate_X(np.pi/2/Npihalf)
    for i in range(Npihalf):
        states.append(dX @ states[-1])
    Nz=20
    dZ = rotate_Z(rot_angle/Nz)
    for i in range(Nz):
        states.append(dZ @ states[-1])
    for i in range(Npihalf):
        states.append(dX @ states[-1])
    states = [Qobj(s) for s in states]
    return states    

In [6]:
class ProgressPlot:
    """
    Real time plot to monitor a QM job
    """
    def __init__(self):
        self.states = None
        with plt.ioff():
            self.fig = plt.figure()
        self.bloch = Bloch(fig=self.fig)
        self.image = widgets.Image(value=self.make_plot(),format='png')
        self.slider_rotangle = widgets.FloatSlider(value=0,min=0,max=1,step=0.1,description='Angle:')
        self.slider_rotangle.observe(self.on_rotangle_change, names='value')
        self.states = ramsey_factory(self.slider_rotangle.value*2*np.pi)
        self.player = widgets.Play(min=0, max=len(self.states)-1, step=1, interval=200, repeat=True)
        self.player.observe(self.play_frame, names='value')
        self.show()

    def make_plot(self):        
        if self.states:
            self.bloch.clear()
            self.bloch.add_states(self.states,'point',colors='C0')
            self.bloch.point_marker='.'
            self.bloch.add_states(self.states[self.player.value % len(self.states)],'vector',colors='C1')
        self.bloch.render()
        s = io.BytesIO()
        self.fig.savefig(s, format="png",bbox_inches='tight', pad_inches=0.5)
        return s.getbuffer()

    def show(self):
        display(self.image,
                widgets.HBox([self.player,self.slider_rotangle,widgets.Label('2π')]),)
        
    def on_rotangle_change(self,change):
        self.states = ramsey_factory(self.slider_rotangle.value*2*np.pi)
        self.player.value = 0
        self.player.max = len(self.states)-1
        
    def play_frame(self,change):
        self.image.value = self.make_plot()
                
pp=ProgressPlot();

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02D\x00\x00\x02D\x08\x06\x00\x00\x00\xf7\xe0;Z\x00\…